In [13]:
# Import packages

import numpy as np
import sys
import pandas as pd
import networkx as nx


import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib
import plotly.io as pio
import seaborn as sns


import aptviz_themes
from supporting import *
pio.templates.default = "aptviz"



In [2]:
## Log

## List of updates needed for ph_outputs:
# [X] read category_orders automatically
# [?] legend title
# [] show sliding window-smoothed birth/death rates
# [] show marginal distributions of birth/deaths on PDs - learned that this doesn't play well with sizing.
#    Might have to use regular subplots for this.
# [] Persistence landscapes?
# [X] Color barcode bars by persistence or #simplices in minimal generator
# [X] Color barcode bars by a node in the minimal generator
# [X] Color barcode bars by a node in killing simplex
# [] Facet plot adjust sizing
# [] Flag for plotting by weight or rank

## Updates that together make a D3 plot
# [] Barcode viz - choose color by dimension, node in minimal generator, etc.

## For fsc viz
#--- Need a color that is NOT associated with dimensions... hmmmm....

In [9]:
## Generate fake data
n_nodes = 50
n_simps = [n_nodes, 100, 80, 20]
max_dim = len(n_simps)-1

fsc_df = create_fake_fsc_df(n_nodes, n_simps, max_dim)
fsc_df.head()

,cell_id,dim,nodes,weight,faces,rank,is_maximal
0,0,0,[0],2.208166,[],3.0,0
1,1,0,[1],2.385766,[],37.0,0
2,2,0,[2],2.057348,[],13.0,1
3,3,0,[3],2.989943,[],16.0,1
4,4,0,[4],2.581098,[],27.0,0


In [ ]:
## Generate fake barcode from fsc USING RANK

n_bars = 100
bar_df = create_fake_barcode(fsc_df, n_bars)
bar_df.head()

In [ ]:
### Color by numeric value - such as persistence or number of nodes/simplices in generator


col_of_interest = "indicator"

bar_df[col_of_interest] = [np.random.rand() for i in np.arange(bar_df.shape[0])]
vmin = np.min(bar_df[col_of_interest])
vmax = np.max(bar_df[col_of_interest])

norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
cmap = matplotlib.cm.get_cmap('cividis')


fig = go.Figure()
bar_df_sorted = bar_df.sort_values(by=["bar_dim", "%%bashr_birth"])
bar_df_sorted = bar_df_sorted.reset_index(drop=True)


for b,bar in bar_df_sorted.iterrows():
    
    if (b==1) :
        showlegendtf = True
    else :
        showlegendtf = False
        

    fig.add_trace(go.Scatter(x=[bar["bar_birth"], bar["bar_death"]], y=[b+1, b+1],
                    mode='lines+markers',
                    marker = dict(size=0.1,
                                  showscale = showlegendtf,
                                  colorscale='cividis',
                                  cmin=0,
                                  cmax=50,
                                  colorbar = dict(title=col_of_interest)),
                    name=f'bar {bar["bar_id"]}',
                    showlegend=False,
                    line=dict(color=f'rgba{cmap(norm(bar[col_of_interest]))}'),
                    hovertemplate =
                        f'<i>id</i>: {bar["bar_id"]}'+
                        f'<br>rep: {bar["rep"]}'+
                        f'<br><b>{col_of_interest}</b>: {bar[col_of_interest]}'))



# Add nice effects and such
fig.update_layout(title_text="Barcode")
fig.update_yaxes(title_text = "H_k")
fig.update_xaxes(title_text = "Filtration index")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(hovermode="x unified")
fig.show()

# fig.write_image("fig1.pdf")

In [ ]:
# Define colors

blue1 = "151a51"
gray1 = "63607c"
red1 = "e14267"
green1 = "a9c4b7"



# From https://github.com/empet/scientific-colorscales/blob/master/scicolorscales.py
lapaz= [[0.0, 'rgb(26, 12, 101)'],
        [0.1, 'rgb(33, 46, 124)'],
        [0.2, 'rgb(39, 77, 145)'],
        [0.3, 'rgb(49, 105, 159)'],
        [0.4, 'rgb(68, 131, 167)'],
        [0.5, 'rgb(102, 153, 164)'],
        [0.6, 'rgb(141, 163, 152)'],
        [0.7, 'rgb(179, 167, 139)'],
        [0.8, 'rgb(223, 183, 148)'],
        [0.9, 'rgb(253, 217, 197)'],
        [1.0, 'rgb(255, 243, 243)']]

davos= [[0.0, 'rgb(44, 26, 76)'],
        [0.1, 'rgb(40, 59, 110)'],
        [0.2, 'rgb(42, 94, 151)'],
        [0.3, 'rgb(68, 117, 193)'],
        [0.4, 'rgb(96, 137, 190)'],
        [0.5, 'rgb(125, 156, 181)'],
        [0.6, 'rgb(155, 175, 172)'],
        [0.7, 'rgb(186, 196, 163)'],
        [0.8, 'rgb(215, 217, 161)'],
        [0.9, 'rgb(237, 236, 206)'],
        [1.0, 'rgb(255, 255, 255)']]

davos_colors = [color[1] for color in davos]

dim0_color = davos[0][1]
dim1_color = davos[1][1]
dim2_color = davos[2][1]
dim3_color = davos[3][1]
dim4_color = davos[4][1]

davos_colors

## Brainstorm

What are some things I might want to know about my data?

### Basics

In [ ]:
fig = px.histogram(fsc_df,x="weight",color="dim", marginal="violin", opacity=0.8)
fig.update_layout(barmode='overlay')

In [ ]:
def fsc_histogram_by_dim(fsc_df, x_col):
    fig = px.histogram(fsc_df,x="weight",color="dim", marginal="violin", opacity=0.8)
    fig.update_layout(barmode='overlay')
    return fig

fig = fsc_histogram_by_dim(fsc_df, x_col)
fig.show()

In [ ]:
# Create fake dataframe of simplices with random dimensions and weights
nSimps = 1000
simp_id = np.arange(nSimps)
simp_dim = np.random.randint(0,10,size=nSimps)
simp_weight = np.random.rand(nSimps)*(simp_dim+1)

df_dev = pd.DataFrame({"simp_id" : simp_id, "weight" : simp_weight, "dim": simp_dim})


df_dev.head()
df_dev.info()
df_dev.describe()
df_dev["dim"].value_counts()
df_dev["dim"]

In [ ]:
px.colors.make_colorscale(davos_colors)

In [ ]:
## Data -- list that records simplex weights or other scalar descriptor
maxdim = 4

data = [np.random.randn(k*50+100) + k for k in np.arange(maxdim)]
# data[k] is an array of weights for k-simplices
# plotcolors = px.colors.continuous.Cividis if maxdim > 11 else [davos_colors[0:maxdim]]
if maxdim > 11 :
    plotcolors = px.colors.sequential.Cividis
else:
    plotcolors = davos_colors[0:maxdim]


In [ ]:
## Plotting
# import plotly.figure_factory as ff

group_labels = [f"{k}" for k in np.arange(maxdim)]



fig = ff.create_distplot(data, group_labels = group_labels, show_curve=True, bin_size = 0.2,
                         colors = davos_colors)


fig.update(layout_title_text= "Distributions of simplex weights",
           layout_yaxis_title = "frequency",
           layout_xaxis_title = "simplex weight",
           layout_legend_title = "simplex<br>dimension",
           layout_plot_bgcolor = "#fafafa")

fig.show()

# Saving works
# fig.write_image("fig1.svg")

In [ ]:
# Using plotly express instead of ff

df = px.data.tips()
fig = px.histogram(df_dev, x="weight",color="dim", marginal="box",histnorm='probability density')
fig.update_layout(yaxis_title="frequency")
fig.show()

# Maybe on hover could light up simplices within the SC.


## Basic persistence diagram and barcode

Nothing fancy. Just what it says.

In [ ]:
# Data FSC -- pandas tidy data frame again so that we can add extra hover stuff later

n_nodes = 50
n_edges = 100
n_2simps = 100
n_3simps = 30

max_filtration = 2
max_dim = 3

def find_nodes(face, df):
    
    nodes = []
    for cell in face:
        nodes.append(df[df["cell_id"]==cell]["nodes"].item())
    return np.unique(nodes)


node_data = np.arange(n_nodes)
cell_id = np.arange(n_nodes)
node_dim = np.zeros(n_nodes)
node_nodes = [[i] for i in np.arange(n_nodes)]
node_weight = np.random.rand(n_nodes)
node_faces = [[] for i in np.arange(n_nodes)]

fsc_df = pd.DataFrame({"cell_id": cell_id, "dim": node_dim, "nodes": node_nodes, "weight": 5+node_weight,
                      "faces": node_faces})


dim1_df = pd.DataFrame({"cell_id": np.arange(n_nodes,n_nodes+n_edges),
                        "dim": np.ones(n_edges),
                        "weight": 4+np.random.rand(n_edges),
                        "faces": [np.random.choice(fsc_df.cell_id, 2) for i in np.arange(n_edges)]})
dim1_df["nodes"] = dim1_df.faces


dim2_df = pd.DataFrame({"cell_id": np.arange(n_nodes+n_edges, n_nodes+n_edges+n_2simps),
                        "dim": 2*np.ones(n_2simps),
                        "weight": 3+np.random.rand(n_2simps),
                        "faces": [np.random.choice(dim1_df.cell_id, 3) for i in np.arange(n_2simps)]})



## NOTE we will likely have 6 nodes per 2-simp because I do not ensure edges actually connect
dim2_df["nodes"] = dim2_df["faces"].apply(find_nodes, df = dim1_df)   


dim3_df = pd.DataFrame({"cell_id": np.arange(n_nodes+n_edges+n_2simps, n_nodes+n_edges+n_2simps+n_3simps),
                        "dim": 3*np.ones(n_3simps),
                        "weight": 2+np.random.rand(n_3simps),
                        "faces": [np.random.choice(dim2_df.cell_id, 4) for i in np.arange(n_3simps)]})



## This method is exploding so just pick some nodes
dim3_df["nodes"] = [np.random.choice(fsc_df.cell_id, 4) for i in np.arange(n_3simps)]  

# Concatenate them all together. Sort and rank weights
fsc = pd.concat([fsc_df, dim1_df, dim2_df, dim3_df])
fsc.dim = fsc.dim.astype(int)
fsc["rank"] = np.argsort(-fsc.weight).astype(float)

# Write fsc to json
fsc.reset_index(inplace=True)
# fsc.to_json("../visualizations/data/fsc_test.json", orient = "records")
fsc.to_csv("../visualizations/data/fsc.csv")

fsc

In [ ]:
# Data -- pandas tidy data frame again so that we can add extra hover stuff later
# Note working only with ranks for ease. All should work fine because I have make the ranks floats :)

n_bars = 100
max_filtration = 2
max_dim = 6

bar_id = np.arange(n_bars)
bar_dim = np.random.randint(0,max_dim,size=n_bars)
bar_birth = np.random.choice(fsc[fsc.dim==1]["rank"],n_bars)
bar_death = np.random.choice(fsc[fsc.dim==2]["rank"],size = n_bars)
bar_rep = [np.random.choice(dim1_df.cell_id, size=4, replace=False) for k in np.arange(n_bars)]
# simp_weight = np.random.rand(nSimps)*(simp_dim+1)

bar_df = pd.DataFrame({"bar_id" : bar_id,
                       "bar_dim" : bar_dim,
                       "bar_birth" : bar_birth,
                       "bar_death" : bar_death,
                       "rep" : bar_rep,
                       "lifetime": bar_death-bar_birth})

bar_df["bar_dim"] = bar_df["bar_dim"].astype(str)

bar_df_sorted = bar_df.sort_values(by=["bar_dim", "bar_birth"])
bar_df_sorted = bar_df_sorted.reset_index(drop=True)

# Write to json
bar_df_sorted.to_json("../visualizations/data/bar_df.json", orient = "records")
bar_df_sorted.to_csv("../visualizations/data/bar_df.csv")

bar_df.head()

In [ ]:
## At this point, we have fake filtered simplicial complex and barcode data that interact as much as they need to.
# All we need to connect the two is the list of simplices associated with a persistent cavity. 

In [ ]:
fig = go.Figure()


for b,bar in bar_df_sorted.iterrows():
    
    if (bar_df_sorted.iloc[b-1]["bar_dim"] != bar["bar_dim"]) :
        showlegendtf = True
    else :
        showlegendtf = False
        
    fig.add_trace(go.Scatter(x=[bar["bar_birth"], bar["bar_death"]], y=[b+1, b+1],
                    mode='lines',
                    line=dict(color=davos_colors[int(bar["bar_dim"])]),
                    name=f'dim {bar["bar_dim"]}',
                    showlegend=showlegendtf,
                    hovertemplate =
                        f'<i>id</i>: {bar["bar_id"]}'+
                        f'<br><b>rep</b>: {bar["rep"]}'))



# Add nice effects and such
fig.update_layout(title_text="Barcode")
fig.update_yaxes(title_text = "H_k")
fig.update_xaxes(title_text = "Filtration index")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(hovermode="x unified")
fig.show()

# # fig.write_image("fig1.pdf")
# #### Ann add lifetime shaded regions. Like this bar is lifetime <20, this is lifetime < 40, etc.


In [ ]:
# Facet them??

fig = px.scatter(bar_df, x="bar_birth", y="bar_death", color="bar_dim", template = "plotly_white",
                 title="Persistence Diagrams (faceted)", hover_data = ["rep"],
                color_discrete_sequence=davos_colors,
                facet_col = "bar_dim", facet_col_wrap=3, 
                 category_orders = {"bar_dim": [str(x) for x in np.arange(max_dim)]})

trace = go.Scatter(
        x=[0, max_filtration],
        y=[0, max_filtration],
        mode="lines",
        line=go.scatter.Line(color= 'rgba(187, 190, 191, .9)', width=0.5),
        showlegend=False)

# give it a legend group and hide it from the legend
trace.update(legendgroup="trendline", showlegend=False)

# add it to all rows/cols, but not to empty subplots
fig.add_trace(trace, row="all", col="all", exclude_empty_subplots=True)

# set only the last trace added to appear in the legend
# `selector=-1` introduced in plotly v4.13
fig.update_traces(showlegend=True)


fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
    row=1, col=1
)




G = nx.random_geometric_graph(200, 0.125)
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))
# Color Node Points
# Color node points by the number of connections.

# Another option would be to size points by the number of connections i.e. node_trace.marker.size = node_adjacencies

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text
# # Create Network Graph
# fig.add_trace(go.Figure(data=[edge_trace, node_trace],
#              layout=go.Layout(
#                 title='<br>Network graph made with Python',
#                 titlefont_size=16,
#                 showlegend=False,
#                 hovermode='closest',
#                 margin=dict(b=20,l=5,r=5,t=40),
#                 annotations=[ dict(
#                     text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
#                     showarrow=False,
#                     xref="paper", yref="paper",
#                     x=0.005, y=-0.002 ) ],
#                 xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
#                 yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
#                 ),
#               row=1,col=2)


fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'),
    row=1,col=2)



fig.add_trace(go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=node_adjacencies,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2)),
             row=1,col=2)




# fig.add_trace(
#     go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
#     row=1, col=2
# )

fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()


In [ ]:
import plotly.graph_objects as go
import datetime
import numpy as np
np.random.seed(1)

programmers = ['Alex','Nicole','Sara','Etienne','Chelsea','Jody','Marianne']

base = datetime.datetime.today()
dates = base - np.arange(180) * datetime.timedelta(days=1)
z = np.random.poisson(size=(len(programmers), len(dates)))

fig = go.Figure(data=go.Heatmap(
        z=z,
        x=dates,
        y=programmers,
        colorscale=lapaz))

fig.update_layout(
    title='GitHub commits per day',
    xaxis_nticks=36)

fig.show()

In [ ]:
import plotly.express as px
df = px.data.iris()
fig = px.parallel_coordinates(df, color="species_id", labels={"species_id": "Species",
                  "sepal_width": "Sepal Width", "sepal_length": "Sepal Length",
                  "petal_width": "Petal Width", "petal_length": "Petal Length", },
                    color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=2)
fig.show()

In [ ]:
import plotly.graph_objects as go

import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/bcdunbar/datasets/master/parcoords_data.csv")

fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = df['colorVal'],
                   colorscale = 'Electric',
                   showscale = True,
                   cmin = -4000,
                   cmax = -100),
        dimensions = list([
            dict(range = [32000,227900],
                 constraintrange = [100000,150000],
                 label = "Block Height", values = df['blockHeight']),
            dict(range = [0,700000],
                 label = 'Block Width', values = df['blockWidth']),
            dict(tickvals = [0,0.5,1,2,3],
                 ticktext = ['A','AB','B','Y','Z'],
                 label = 'Cyclinder Material', values = df['cycMaterial']),
            dict(range = [-1,4],
                 tickvals = [0,1,2,3],
                 label = 'Block Material', values = df['blockMaterial']),
            dict(range = [134,3154],
                 visible = True,
                 label = 'Total Weight', values = df['totalWeight']),
            dict(range = [9,19984],
                 label = 'Assembly Penalty Wt', values = df['assemblyPW']),
            dict(range = [49000,568000],
                 label = 'Height st Width', values = df['HstW'])])
    )
)
fig.show()

In [ ]:
nSimps = 1000
simp_id = np.arange(nSimps)
simp_weight = np.random.rand(nSimps)
simp_dim = np.random.randint(0,10,size=nSimps)

df_dev = pd.DataFrame({"simp_id" : simp_id, "weight" : simp_weight, "dim": simp_dim})
# df_dev = {"simp_id" : np.arange(1,nSimps), "weight" : np.random.rand(nSimps)}

df_dev


In [ ]:
import plotly.express as px
df = px.data.election()
fig = px.scatter_ternary(df, a="Joly", b="Coderre", c="Bergeron", hover_name="district",
    color="winner", size="total", size_max=15,
    color_discrete_map = {"Joly": "blue", "Bergeron": "green", "Coderre":"red"} )
fig.show()

In [ ]:
df